## Loading inmunofluorescence images

In [1]:
!wget http://nanostring-public-share.s3-website-us-west-2.amazonaws.com/GeoScriptHub/KidneyDataset/ROI_reports.zip
!unzip -q ROI_reports.zip
!unzip -q "ROI reports/*.zip"

--2021-04-16 19:14:05--  http://nanostring-public-share.s3-website-us-west-2.amazonaws.com/GeoScriptHub/KidneyDataset/ROI_reports.zip
Resolving nanostring-public-share.s3-website-us-west-2.amazonaws.com (nanostring-public-share.s3-website-us-west-2.amazonaws.com)... 52.218.178.235
Connecting to nanostring-public-share.s3-website-us-west-2.amazonaws.com (nanostring-public-share.s3-website-us-west-2.amazonaws.com)|52.218.178.235|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2500959921 (2.3G) [application/x-zip-compressed]
Saving to: ‘ROI_reports.zip’

ROI_reports.zip     100%[===================>]   2.33G  37.2MB/s    in 69s     

2021-04-16 19:15:14 (34.7 MB/s) - ‘ROI_reports.zip’ saved [2500959921/2500959921]


7 archives were successfully processed.


## Crop to ROI

In [189]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from os import path,mkdir,listdir
import re
from tqdm import tqdm

def cropToRoi(image):
    "input: np.uint8 (H,W,3)"
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    binary = cv2.inRange(gray,250,255)

    #Find contours
    contours, _ = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if contours is None:
        raise 'Contours not found'
    #Find largest contour
    contour = max(contours, key=cv2.contourArea)
    #Draw mask from largest contour
    mask = np.zeros_like(image)
    mask = cv2.drawContours(mask,[contour],0,(255,255,255),-1)

    #Apply mask
    out = image*(mask>0)

    #Crop to minimum enclosing rectangle
    x,y,w,h = cv2.boundingRect(contour)
    out = out[y:y+h,x:x+w]
    return out
  
def batch_job(filenames,output_dir,function):
  extensions = ['.png']
  if not path.isdir(output_dir):
    mkdir(output_dir)

  for filename in tqdm(filenames):

    if path.splitext(filename)[1] in extensions:
      sample = cv2.imread(filename)
      out = function(sample)

      output_filename = path.join(output_dir,path.split(filename)[1])

      cv2.imwrite(output_filename,out)
    else:
      continue
    
  return None

In [175]:
pattern = r'\d\d\d.png'
input_dir = "/content"
output_dir = "/content/roi_crops"

filenames = [f for f in listdir(input_dir) if re.search(pattern,f)]

batch_job(filenames,output_dir,cropToRoi)

100%|██████████| 211/211 [00:35<00:00,  5.89it/s]


## Extract features

In [230]:
from torchvision.models import resnet50
from torchvision.datasets import ImageFolder
from torchvision import io
import torchvision.transforms as T
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

class ScanDataset(Dataset):
  def __init__(self,filenames,transform):
    self.filenames = filenames
    self.transform = transform
  def __len__(self):
    return len(self.filenames)
  def __getitem__(self,idx):
    image = io.read_image(filenames[idx],io.image.ImageReadMode.RGB)/255.0

    image = self.transform(image)
    return image

@torch.no_grad()
def get_features(dataloader):
  features = []
  for Images in dataloader:
    out = model(Images)
    features.append(out)
  features = torch.cat(features,axis=0).cpu().numpy()
  return features


In [235]:
#DEFINITIONS
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = resnet50(pretrained=True)
model = model.to(device).eval()
model.fc = nn.Identity()

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
transform = T.Compose([T.Resize(512),T.Normalize(mean,std)])

dataset = ScanDataset(filenames,transform)
dataloader = DataLoader(dataset,batch_size=1,shuffle=False)
#ONly batch of 1 supported

In [239]:
features = get_features(dataloader)

In [250]:
import pandas as pd
df = pd.DataFrame(features)
df["ROI"] = filenames
df = df.set_index('ROI')

In [257]:
df.to_csv('ROI_spatial_features_pretrained.csv',index=True)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
ROI,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
normal3_scan - 031.png,0.484706,0.509939,0.129336,0.044924,0.257169,0.164471,0.259097,0.473763,0.465042,0.510889,0.487447,0.085665,0.346571,0.126790,0.249936,0.111845,0.073683,0.715550,0.213034,0.360463,0.593203,1.707684,0.187793,0.120260,0.511338,0.267202,0.065508,0.187592,0.734347,0.079620,0.624104,0.449268,0.156668,0.209994,0.541730,0.036290,0.031538,0.304601,0.678049,0.378840,...,0.091436,0.106500,0.267741,0.097787,0.171669,0.246501,0.143365,0.646787,0.555773,0.302362,0.352308,0.937950,0.130751,0.106181,0.635648,1.817431,1.069847,0.056892,0.376327,0.243830,1.146830,0.366514,0.087723,0.184106,0.920520,0.585554,0.269690,0.081310,0.322473,0.149548,0.310641,0.324107,0.051701,0.171758,0.252338,0.311830,0.242641,0.289326,0.174760,0.404002
normal3_scan - 024.png,0.343536,0.244414,0.151053,0.075289,0.106447,0.150362,0.120152,0.631196,0.459705,0.435167,0.464323,0.047940,0.594783,0.123692,0.196179,0.113424,0.042091,0.862080,0.293256,0.578698,0.712649,1.759398,0.216236,0.172612,0.366369,0.135880,0.089632,0.220986,0.507509,0.090999,0.657405,0.381630,0.274366,0.328094,0.644227,0.053125,0.033152,0.415117,0.599363,0.356963,...,0.076371,0.184065,0.307635,0.171157,0.133332,0.298794,0.133671,0.306910,0.330662,0.226133,0.500237,0.996091,0.213661,0.076978,0.474528,1.504833,1.071918,0.042994,0.418886,0.141816,1.058818,0.527671,0.050386,0.291644,1.313901,1.012172,0.195628,0.096886,0.369191,0.139077,0.287084,0.422399,0.049061,0.091039,0.204695,0.255616,0.220233,0.203673,0.115541,0.429592
disease4_scan - 011.png,0.176480,0.310755,0.245659,0.115493,0.132630,0.099644,0.180676,0.180323,0.428611,0.311114,0.299459,0.064472,0.643635,0.272603,0.289662,0.323711,0.085620,0.674939,0.529160,0.599199,0.403080,1.435189,0.230336,0.072477,0.629306,0.458202,0.166190,0.309423,0.318884,0.153174,0.535455,0.512155,0.136957,0.512339,1.025527,0.043467,0.028823,0.524344,0.603228,0.364343,...,0.159760,0.078013,0.297313,0.263266,0.257720,0.177868,0.188600,0.251292,0.442871,0.206270,0.378321,0.677145,0.278854,0.079668,0.141433,1.336608,0.860430,0.058808,0.395905,0.218924,0.619567,0.299880,0.100728,0.169969,0.688185,0.778717,0.365745,0.046757,0.229418,0.093363,0.142539,0.102919,0.075196,0.104389,0.063497,0.202875,0.214053,0.408015,0.193005,0.615288
normal3_scan - 025.png,0.377301,0.246311,0.075803,0.069532,0.125852,0.134769,0.311540,0.335114,0.374974,0.379843,0.347797,0.024824,0.508925,0.097144,0.265047,0.138126,0.022016,0.497194,0.232268,0.374180,0.564251,1.405967,0.380121,0.161116,0.257952,0.082720,0.179397,0.148387,0.482330,0.103142,0.547385,0.310079,0.198558,0.224741,0.555569,0.050356,0.026397,0.461798,0.569553,0.280543,...,0.076000,0.248163,0.275676,0.162352,0.213658,0.272480,0.282383,0.343585,0.524347,0.176000,0.515390,0.671101,0.175710,0.113449,0.402310,1.148597,0.968581,0.057944,0.351891,0.261359,1.389356,0.273366,0.045547,0.143150,0.737845,0.659060,0.206256,0.061998,0.371348,0.091244,0.296996,0.288001,0.128925,0.131137,0.246647,0.299439,0.136277,0.158915,0.233173,0.330438
normal3_scan - 043.png,0.298395,0.229861,0.168687,0.071691,0.213942,0.211259,0.246359,0.344468,0.312769,0.424126,0.324062,0.028141,0.391812,0.146089,0.188574,0.072709,0.079399,0.569358,0.253870,0.197340,0.642249,1.453377,0.301437,0.124704,0.401232,0.248246,0.052702,0.211516,0.637556,0.020238,0.539126,0.361324,0.145771,0.296292,0.538837,0.037031,0.022915,0.397266,0.535787,0.385366,...,0.047209,0.161708,0.194248,0.043108,0.113257,0.251138,0.195875,0.403115,0.600305,0.249940,0.230000,0.969349,0.161130,0.086842,0.364030,1.359819,0.929231,0.026446,0.372625,0.192800,0.994462,0.380856,0.080851,0.181308,0.901475